# aotelem
This notebook pulls down data for raw IFS frames. Only images that were successfully processed to "reduced data products" are considered.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import time
import astropy.time as aptime
import copy
import warnings
import pandas as pd
import os

import gpiTools as gpt

In [2]:
%matplotlib inline
pylab.rcParams['figure.figsize'] = (14.0, 5.0)  

## query DB. Seeing entries will need cleaning

In [3]:
querydbflag = True

In [4]:
if querydbflag:
    import gpifilesdb
    db = gpifilesdb.GPIFilesDB(server='127.0.0.1', username='mtallis')

This cell contains data that matches up with AO telemetry. Used to calibrate the WFE values

In [6]:
if 'dbres' in locals():
    del(dbres)
        
savedir = os.environ["HOME"]+'/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/code/'

if querydbflag:
    query = """SELECT
    AORAW_FILES.datafile, objname,whenstr from AORAW
    left join AORAW_FILES on AORAW.UID = AORAW_FILES.RAWID
    where objname not like 'Zenith' and
    aoframes like 1000 and
    datafile like '%_phase%' order by whenstr

    """
    
    # the directory stuff at the end is b/c there are a 
    # couple duplicate folders with permutations of the same name
    # and the data got stored in both (eg: HD 75519 and HD_75519)
    # Get rid of the non-standard directory names


    dbres = db._do_query(query)
    dbdf = pd.DataFrame(dbres)
    
    print 'outputting dataframe to '+savedir
    dstr = time.strftime('%Y%m%d')
    prefix = 'aotelem_'
    dbdf.to_excel(savedir+prefix+dstr+'.xls', index=False)
    dbdf.to_csv(savedir+prefix+dstr+'.txt', index=False)

    
else:
    dstr = '20190226'
    print "******************************************************************"
    print "Loading DB query w/ dirty seeing queried on "+dstr
    print "******************************************************************"

    dbdf = pd.read_csv(savedir+'aotelem_'+dstr+'.txt')
    
print "There are %g entries.\n"%len(dbdf)

outputting dataframe to /Users/MelisaT/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/code/
There are 2009 entries.



This does not link to the AO telemetry data. Only contains IFS data

In [8]:
if 'dbres' in locals():
    del(dbres)
        
savedir = os.environ["HOME"]+'/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/code/'

if querydbflag:
    query = """SELECT
    RawDataProducts.DATAFILE, DATALAB, OBJNAME,
    DATESTR, UTSTART, MJDOBS, ITIME, COADDS,
    OBSMODE, DISPERSR, IFSFILT, AOFRAMES, AOSPATIA,
    HMAG, STMAG as IMAG, FLUX as AOFLUX, AOWFE as RawDPwfe, 
    
    PAR_ANG, PA, IAA, AZIMUTH, ELEVATIO, AIRMASS, AMSTART, AMEND,
    WINDM2, WINDM2DR, WINDDIRE, WINDSPEE, TAMBIENT, OMSATEMP, GLITEMP, GLOTEMP,
    MASSSEE, MASSTAU, DIMMSEE,
    MASS05CN, MASS1CN2, MASS2CN2, MASS4CN2, MASS8CN2, MASS16CN, MASSISOP,    
    
    DRPDATE,
    ReducedDataProducts.CONTR025, ReducedDataProducts.CONTR040, ReducedDataProducts.CONTR080
    
    from RawDataProducts
    left join OBSNOTES on RawDataProducts.UID = OBSNOTES.UID
    inner join Raw2Reduced on RawDataProducts.UID = Raw2Reduced.RAWID
    inner join ReducedDataProducts on Raw2Reduced.REDID = ReducedDataProducts.UID
    
    where RawDataProducts.OBJNAME not like 'Zenith'
    and RawDataProducts.ARTSRC like 'GPI_DEPLOY_OUT'
    and RawDataProducts.AOCLOOP like 'TRUE'
    and RawDataProducts.DATAFILE like 'S201%S%.fits'
    and OBSNOTES.markedbad is NULL

    and ReducedDataProducts.DATAFILE like 'S201%S%_spdc_distorcorr.fits'
    and ReducedDataProducts.DROPBOXPATH like '%autoreduced%'
    and ReducedDataProducts.DROPBOXPATH not like '%Non-Campaign%'
    and ReducedDataProducts.filetype = 'Spectral Cube'

    and ReducedDataProducts.DROPBOXPATH not like '%/HR_4796/%'
    and ReducedDataProducts.DROPBOXPATH not like '%/HD 75519/%'
    
    order by RawDataProducts.DATAFILE , DRPDATE DESC    
    """

    # the directory stuff at the end is b/c there are a 
    # couple duplicate folders with permutations of the same name
    # and the data got stored in both (eg: HD 75519 and HD_75519)
    # Get rid of the non-standard directory names


    dbres = db._do_query(query)
    dbdf = pd.DataFrame(dbres)
    del(dbres)
    dbdf['MJDOBS'] = pd.to_numeric(dbdf['MJDOBS'])
    
    print 'outputting dataframe to '+savedir
    dstr = time.strftime('%Y%m%d')
    prefix = 'IFS_AllOnSky_RawDistorcorr_'
    dbdf.to_excel(savedir+prefix+dstr+'.xls', index=False)
    dbdf.to_csv(savedir+prefix+dstr+'.txt', index=False)

    
else:
    dstr = '20190226'
    print "******************************************************************"
    print "Loading DB query w/ dirty seeing queried on "+dstr
    print "******************************************************************"

    dbdf = pd.read_csv(savedir+'IFS_AllOnSky_RawDistorcorr_'+dstr+'.txt')
    
print "There are %g entries.\n"%len(dbdf)

outputting dataframe to /Users/MelisaT/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/code/
There are 27611 entries.



In [17]:
print dbdf.dtypes

DATAFILE     object
DATALAB      object
OBJNAME      object
DATESTR      object
UTSTART      object
MJDOBS      float64
ITIME       float64
COADDS        int32
OBSMODE      object
DISPERSR     object
IFSFILT      object
AOFRAMES      int32
AOSPATIA    float64
HMAG        float64
IMAG        float64
AOFLUX      float64
RawDPwfe    float64
PAR_ANG     float64
PA          float64
IAA         float64
AZIMUTH     float64
ELEVATIO    float64
AIRMASS     float64
AMSTART     float64
AMEND       float64
WINDM2      float64
WINDM2DR    float64
WINDDIRE    float64
WINDSPEE    float64
TAMBIENT    float64
OMSATEMP    float64
GLITEMP     float64
GLOTEMP     float64
MASSSEE     float64
MASSTAU     float64
DIMMSEE     float64
MASS05CN    float64
MASS1CN2    float64
MASS2CN2    float64
MASS4CN2    float64
MASS8CN2    float64
MASS16CN    float64
MASSISOP    float64
DRPDATE      object
CONTR025    float64
CONTR040    float64
CONTR080    float64
dtype: object


## Check for multiply-matched raw data

In [9]:
if len(set(dbdf['DATAFILE'])) < len(dbdf):
    print 'number of files = '+str(len(dbdf))
    print 'number of unique= '+str(len(set(dbdf['DATAFILE'])))
    #raise Exception('Some raw files have multiple entries. Quitting so you can figure this out.')
    
    f0 = ''
    ct = 0
    df1 = pd.DataFrame(dbdf[ct:ct+1], index=[0])

    for f in dbdf['DATAFILE']:
        if f == f0:
            df1 = df1.append(dbdf[ct:ct+1], ignore_index=False)
        f0 = f
        ct = ct+1

    df1.drop(0,inplace=True) # because it was a dummy entry
    
    print df1[['DATAFILE','CONTR040','OBJNAME']]
    
    raise Exception('Duplicate raw entries. You figure out what to do.')

else:
    print 'Hurray! All raw files have only 1 reduced match.'

number of files = 27611
number of unique= 27606
                  DATAFILE  CONTR040          OBJNAME
24788  S20180811S0166.fits       NaN  TYC 8785-1322-1
24807  S20180811S0185.fits  0.000270  TYC 8785-1322-1
24843  S20180811S0223.fits  0.000123           BO Mic
24845  S20180811S0224.fits  0.000134           BO Mic
24870  S20180811S0251.fits  0.000072           mu PsA


Exception: Duplicate raw entries. You figure out what to do.

# Remove bad seeing values 
- 0
- repeated for >=30min

In [9]:
# remember: in DATESTR we use the UT date that covers most
#  of the night (everthing from ~3pm local time on is recorded with the next date)
# the "datetimes" reconstruction from MJD is the right one!

t = aptime.Time(dbdf['MJDOBS'], format='mjd')
#dbdf['datetime'] = t.datetime
#dbdf[['DATESTR','UTSTART','datetime']][270:280]

In [10]:
seeing_vars = ['DIMMSEE','MASSSEE','MASSTAU','MASSISOP','MASS05CN',\
               'MASS1CN2','MASS2CN2','MASS4CN2','MASS4CN2','MASS8CN2','MASS16CN']
seeing_is_invalid = {}
for var in seeing_vars:
    # remove repeats
    seeing_is_invalid[var] = gpt.find_invalid_repeats(dbdf[var],t.datetime,30)
    dbdf.loc[seeing_is_invalid[var],var] = np.nan
    
    # remove 0s, if present
    try:
        dbdf.loc[dbdf[var]==0, var] = np.nan
    except:
        continue

In [12]:
dbdf = dbdf.drop_duplicates(subset='DATAFILE')

In [13]:
savedir = os.environ["HOME"]+'/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/code/'
prefix = 'IFS_AllOnSky_RawDistorcorr_CleanSee30_'
#prefix = 'AO_telem_'
dbdf.to_excel(savedir+prefix+dstr+'.xls', index=False)
dbdf.to_csv(savedir+prefix+dstr+'.txt', index=False)